<a href="https://colab.research.google.com/github/CFT-EPN/actividades_club/blob/main/juego_vida.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

**Actividad organizada por:**



[CFT-EPN](https://es-la.facebook.com/CFTepn/)

Elaborado por: Luis Palacios

## **LIBRERÍAS**

In [ ]:
import numpy as np

import matplotlib.pyplot as plt
from matplotlib import animation #necesario para la animacion
from matplotlib.animation import PillowWriter #necesario para hacer un gif


In [ ]:
#Necesario para hacer videos en mp4
# pip install ffmpeg-python

## **PROGRAMA Y SIMULACIÓN**

In [ ]:
"""--------------------------------------------------------------------------------------
Luis Palacios, julio 2022
Cambios por Daniel García-Villacañas, enero 2023
Juego de la vida de Conway
Estado de celdas: viva (1), muerta (0)
Reglas:  
1. Cada celda viva sobrevivirá si 2 o 3 celdas vecinas están vivas. 
   En caso contrario muere.
2. Cada celda con exactamente 3 celdas vecinas vivas "resucita". 
   Caso contrario continúa muerta.   

Se utilizan condiciones de frontera fijas tq. todas las celdas externas están muertas
--------------------------------------------------------------------------------------"""
def game(matrix):
  #matriz extendida, condiciones de frontera tq. todas las celdas externas están muertas
  Np, Mp = matrix.shape  
  N, M = Np-2, Mp-2
  
  #matriz auxiliar que lleva las operaciones sobre cada celda de la matriz interna
  aux = np.zeros((Np, Mp), dtype=int) 
  for i in range(1,N+1):
    for j in range(1,M+1):
      aux = rules(matrix, i, j, aux)
  return aux

#Reglas dadas por Conway: pueden modificarse como quieran
def rules(matrix, row, column, aux):

  state = matrix[row, column]
  #conteo de las celdas vivas alrededor de (row, column)
  counter = 0
  for i in range(row-1, row+2):
    for j in range(column-1, column+2):
      if(matrix[i, j] == 1): 
        counter = counter + 1
  
  #restamos el estado de la celda (row, column)
  counter = counter - state

  if((state==1) and ((counter==2) or (counter==3))):
    aux[row, column] = 1
  elif((state==0) and (counter==3)):
    aux[row, column] = 1
  else:
    aux[row, column] = 0
  return aux

#verifica que entre dos iteraciones consecutivas no se tenga la misma forma
def stationarity(matrix, aux):
  Np, Mp = matrix.shape
  N, M = Np-2, Mp-2
  check = 0 
  for i in range(1,N+1):
    for j in range(1, M+1):
      if(matrix[i, j] == aux[i, j]):
        check = check + 1
  if(check == (N*M)): #si todos los valores son iguales check = 1
    check = 1
  else:
    check = 0
  return check

#Inicializa la matriz con las condiciones de frontera 
def set_matrix():
  print("Ingrese el número de filas N y columnas M de las celdas del Juego de la Vida")
  N = int(input('N: '))
  M = int(input('M: '))
  matrix = np.zeros((N+2, M+2), dtype=int)
  return matrix, N, M

#Define 3 posibles formas de establecer las condiciones iniciales a la matriz
def initial_state(option):
  #Inicialización de la semilla de los números aleatorios
  np.random.seed(0) 
  
  #Condiciones aleatorias de parámetro p
  if(option == 1):
    matrix, N, M = set_matrix()
    p = float(input("Ingrese el valor límite p [0, 1] en el que una celda está viva (w>p celda muerta): "))
    for i in range(1, N+1):
      for j in range(1, M+1):
        w = np.random.rand()
        if(w<p):
          matrix[i,j] = 1

  #Ingreso manual
  elif(option == 2):
    matrix, N, M = set_matrix()
    for i in range(1, N+1):
      print('Fila {}'.format(i))
      #row = [int(input()) for j in range(1, M+1)]
      rowstr=input()
      row =[int(num) for num in rowstr]
      matrix[i, 1:M+1] = row
      print(row)
    print('Las celdas iniciales ya se han fijado')
  
  #Condiciones conocidas
  elif(option == 3):
    print('Posibles casos')
    print('(1). The R-pentomino (5x5)')
    print('(2). Diehard (5x10)')
    print('(3). Acorn (5x9)')
    print('(4). Glider (5x5)')
    print('(5). Middle-weight spaceship (8x8)')

    choice = int(input("Elección: "))

    print("Ingrese el número de filas N y columnas M de las celdas del Juego de la Vida")
    N = int(input('N: '))
    M = int(input('M: '))
    print("Número de filas y columnas expandidas,(Ns, Ms) != (N, M)")
    Ns = int(input('Ns > N: '))
    Ms = int(input('Ms > M: '))
    
    if((Ns<= N) or (Ms<= M)): 
      print('Ns<= N o Ms<=M')
      # break
    matrix = np.zeros((Ns+2, Ms+2), dtype=int)
    
    #centra la matriz
    M1 = int(0.5*(Ms-M))
    M2 = int(0.5*(Ms+M))
    N1 = int(0.5*(Ns-N))

    if(choice == 1):
      matrix[N1-1+1,M1:M2]=[0,0,0,0,0]
      matrix[N1-1+2,M1:M2]=[0,0,1,1,0]
      matrix[N1-1+3,M1:M2]=[0,1,1,0,0]
      matrix[N1-1+4,M1:M2]=[0,0,1,0,0]
      matrix[N1-1+5,M1:M2]=[0,0,0,0,0]
    elif(choice == 2):
      matrix[N1-1+1,M1:M2]=[0,0,0,0,0,0,0,0,0,0]
      matrix[N1-1+2,M1:M2]=[0,0,0,0,0,0,0,1,0,0]
      matrix[N1-1+3,M1:M2]=[0,1,1,0,0,0,0,0,0,0]
      matrix[N1-1+4,M1:M2]=[0,0,1,0,0,0,1,1,1,0]
      matrix[N1-1+5,M1:M2]=[0,0,0,0,0,0,0,0,0,0]
    elif(choice == 3):
      matrix[N1-1+1,M1:M2]=[0,0,0,0,0,0,0,0,0]
      matrix[N1-1+2,M1:M2]=[0,0,1,0,0,0,0,0,0]
      matrix[N1-1+3,M1:M2]=[0,0,0,0,1,0,0,0,0]
      matrix[N1-1+4,M1:M2]=[0,1,1,0,0,1,1,1,0]
      matrix[N1-1+5,M1:M2]=[0,0,0,0,0,0,0,0,0]     
    elif(choice == 4):
      matrix[N1-1+1,M1:M2]=[0,0,0,0,0]
      matrix[N1-1+2,M1:M2]=[0,1,0,1,0]
      matrix[N1-1+3,M1:M2]=[0,0,1,1,0]
      matrix[N1-1+4,M1:M2]=[0,0,1,0,0]
      matrix[N1-1+5,M1:M2]=[0,0,0,0,0]    
    elif(choice == 5):
      matrix[N1-1+1,M1:M2]=[0,0,0,0,0,0,0,0]
      matrix[N1-1+2,M1:M2]=[0,0,0,1,0,0,0,0]
      matrix[N1-1+3,M1:M2]=[0,1,0,0,0,1,0,0]
      matrix[N1-1+4,M1:M2]=[0,0,0,0,0,0,1,0]
      matrix[N1-1+5,M1:M2]=[0,1,0,0,0,0,1,0]
      matrix[N1-1+6,M1:M2]=[0,0,1,1,1,1,1,0]
      matrix[N1-1+7,M1:M2]=[0,0,0,0,0,0,0,0] 
      matrix[N1-1+8,M1:M2]=[0,0,0,0,0,0,0,0]
    else:
      print("Elección no válida")
      # break
    N, M = Ns, Ms
  
  return matrix, N, M

#----------------------------------------------------------------------------------------
#Parte principal del programa
#----------------------------------------------------------------------------------------
print('Elija la forma de establecer las condiciones iniciales en el Juego de la Vida')
print("1. Condiciones aleatorias de parámetro p")
print('2. Ingreso manual')
print('3. Condiciones conocidas')

option = int(input("Elección: "))

#almacena todos los estados d1e las celdas en cada iteración
matrix_states = []

#condiciones iniciales
matrix, N, M = initial_state(option)

#guardamos la matriz interna que contiene la dinámica
matrix_states.append(matrix[1:N+1, 1:M+1])

T = int(input("Número de iteraciones del Juego de la Vida: "))

for it in range(1, T+1):
  aux = matrix
  #juego con las reglas definidas
  matrix = game(matrix)
  #comprobación de estado estacionario entre dos iteraciones seguidas
  check = stationarity(matrix, aux)
  if(check == 1):
    print("Estado estacionario en {} iteraciones".format(it-1))
    break
  #salvado de cada iteración
  matrix_states.append(matrix[1:N+1, 1:M+1])

print("La simulación terminó.")
print("Ahora pasaremos a la animación.")

Elija la forma de establecer las condiciones iniciales en el Juego de la Vida
1. Condiciones aleatorias de parámetro p
2. Ingreso manual
3. Condiciones conocidas
Elección: 3
Posibles casos
(1). The R-pentomino (5x5)
(2). Diehard (5x10)
(3). Acorn (5x9)
(4). Glider (5x5)
(5). Middle-weight spaceship (8x8)
Elección: 2
Ingrese el número de filas N y columnas M de las celdas del Juego de la Vida
N: 5
M: 10
Número de filas y columnas expandidas,(Ns, Ms) != (N, M)
Ns > N: 10
Ms > M: 15
Número de iteraciones del Juego de la Vida: 100
Estado estacionario en 56 iteraciones
La simulación terminó.
Ahora pasaremos a la animación.


## **MONTAMOS EL DRIVE**

In [ ]:
#montar google drive
from google.colab import drive
drive.mount('/content/gdrive') 
path = '/content/gdrive/MyDrive/'

Mounted at /content/gdrive


## **ANIMACIÓN**

In [ ]:
# init the figure
fig, ax = plt.subplots(figsize=(10,8))

def update(k):
    #borramos cualquier cosa en ax
    ax.clear()

    #graficamos el estado k de la simulación 
    p = ax.imshow(matrix_states[k], cmap='Greys', interpolation='nearest')

    #configuración visual
    xticks = np.arange(-0.5, M-0.5)
    yticks = np.arange(-0.5, N-0.5)
    ax.set_yticks(yticks)
    ax.set_xticks(xticks)
    ax.set_title('Game of Life', fontsize=18)
    plt.text(M, -1, r'$it = {}$'.format(k), fontsize=15)
    ax.xaxis.set_ticklabels([])
    ax.yaxis.set_ticklabels([])
    plt.grid(linewidth = 1, color='black')

ani = animation.FuncAnimation(fig, update, frames=len(matrix_states), interval=100)

#se puede hacer un video en mp4
# writervideo = animation.FFMpegWriter(fps=5) 
# ani.save(path+'animation.mp4', writer=writervideo)

#se puede hacer un gif
ani.save(path+'animation.gif', writer='pillow', fps=1)

plt.close(fig)

In [ ]:
print("La animación está lista y se puede visualizar en la carpeta del programa.")